# Compare the results of different posterior results

https://python.arviz.org/en/latest/api/generated/arviz.compare.html

In [ ]:
import arviz as az
from elisa.infer.fit import PosteriorResult

f_bay_result_Compt = PosteriorResult.load('/mnt/d/temp/Part10_posresul_Compt.pkl.xz', decompress='lzma')
f_bay_result_BlackbodyRad = PosteriorResult.load('/mnt/d/temp/Part10_posresul_BlackbodyRad.pkl.xz', decompress='lzma')
f_bay_result_OTTB = PosteriorResult.load('/mnt/d/temp/Part10_posresul_OTTB.pkl.xz', decompress='lzma')
# 
res_dic = {
    'Compt': f_bay_result_Compt.idata,
    'BlackbodyRad': f_bay_result_BlackbodyRad.idata,
    'OTTB': f_bay_result_OTTB.idata,
}

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [28]:
compare_res_waic = az.compare(res_dic, ic='waic', var_name='channels', scale='deviance')
compare_res_waic

/home/meotor/anaconda3/lib/python3.11/site-packages/arviz/stats/stats.py:1632: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(
/home/meotor/anaconda3/lib/python3.11/site-packages/arviz/stats/stats.py:1632: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


,rank,elpd_waic,p_waic,elpd_diff,weight,se,dse,warning,scale
Compt,0,22.973980,2.378177,0.000000,9.347593e-01,7.476891,0.000000,True,deviance
BlackbodyRad,1,26.320752,1.988522,3.346772,6.528608e-15,5.782254,4.144575,False,deviance
OTTB,2,55.976882,4.658357,33.002901,6.524067e-02,10.242586,12.111212,True,deviance


In [30]:
compare_res_loo = az.compare(res_dic, ic='loo', var_name='channels', scale='deviance')
print(compare_res_loo)

              rank   elpd_loo     p_loo  elpd_diff        weight         se  \
Compt            0  23.079444  2.430909   0.000000  9.338751e-01   7.512390   
BlackbodyRad     1  26.380073  2.018182   3.300629  2.857125e-15   5.787652   
OTTB             2  56.153853  4.746843  33.074409  6.612494e-02  10.276691   

                    dse  warning     scale  
Compt          0.000000    False  deviance  
BlackbodyRad   4.178895    False  deviance  
OTTB          12.166659    False  deviance  


# Combine posterior results from different (time-resolved) spectra

In [1]:
import arviz as az
import xarray as xr
import numpy as np
from elisa.infer.fit import PosteriorResult

def combine_idata(list_pos_result):
    """
    Combines multiple InferenceData objects and computes the mean of reff values.
    
    This function takes a list of PosteriorResult objects, each containing an `idata` 
    and a `reff` attribute. It calculates the mean of the `reff` values, determines 
    the global minimum length of the "draw" dimension across all datasets, trims all 
    datasets to this minimum length, and concatenates them along a new "channel" dimension.
    
    Parameters:
    - list_pos_result (list): A list of PosteriorResult objects, each having an `idata` 
      (InferenceData) and a `reff` attribute.
    
    Returns:
    - combined_idata (InferenceData): The concatenated InferenceData object with a new 
      "channel" dimension.
    - mean_reff (float): The mean value of the `reff` attributes from the input results.
    """

    # Calculate the mean of the reff values from the input results
    mean_reff = np.mean([r.reff for r in list_pos_result])
    
    # Extract the idata objects from the input results
    list_idata = [r.idata for r in list_pos_result]
    
    # Determine the global minimum length of the "draw" dimension across all groups
    global_min_draw = float("inf")
    for idata in list_idata:
        for group in idata.groups():
            ds = getattr(idata, group)
            if ds is None or "draw" not in ds.dims:
                continue
            current_draw = ds.sizes["draw"]
            if current_draw < global_min_draw:
                global_min_draw = current_draw

    # Trim all datasets to the global minimum draw length
    trimmed_idatas = []
    for idata in list_idata:
        trimmed_groups = {}
        for group in idata.groups():
            ds = getattr(idata, group)
            if ds is None:
                continue
            if "draw" in ds.dims:
                trimmed_ds = ds.isel(draw=slice(None, global_min_draw))
            else:
                trimmed_ds = ds
            trimmed_groups[group] = trimmed_ds
        trimmed_idata = az.InferenceData(**trimmed_groups)
        trimmed_idatas.append(trimmed_idata)

    # Concatenate the trimmed datasets along a new "channel" dimension
    combined_idata = az.InferenceData(
        posterior_predictive=xr.concat(
            [d.posterior_predictive[["channels"]] for d in trimmed_idatas],
            dim="channel",
        ),
        log_likelihood=xr.concat(
            [d.log_likelihood[["channels"]] for d in trimmed_idatas], dim="channel"
        ),
        observed_data=xr.concat(
            [d.observed_data[["channels"]] for d in trimmed_idatas], dim="channel"
        ),
    )
    return combined_idata, mean_reff


In [ ]:
f_bay_result_Compt_p1 = PosteriorResult.load('/mnt/d/temp/Part01_posresul_Compt.pkl.xz', decompress='lzma')
f_bay_result_Compt_p2 = PosteriorResult.load('/mnt/d/temp/Part02_posresul_Compt.pkl.xz', decompress='lzma')
idata_Compt_p1 = f_bay_result_Compt_p1.idata
idata_Compt_p2 = f_bay_result_Compt_p2.idata
list_pos_result_Compt = [f_bay_result_Compt_p1, f_bay_result_Compt_p2]
combined_idata_Compt, mean_reff_Compt = combine_idata(list_pos_result_Compt)
combined_idata_Compt

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Inference data with groups:
	> posterior_predictive
	> log_likelihood
	> observed_data

In [ ]:
f_bay_result_BlackbodyRad_p1 = PosteriorResult.load('/mnt/d/temp/Part01_posresul_BlackbodyRad.pkl.xz', decompress='lzma')
f_bay_result_BlackbodyRad_p2 = PosteriorResult.load('/mnt/d/temp/Part02_posresul_BlackbodyRad.pkl.xz', decompress='lzma')
idata_BlackbodyRad_p1 = f_bay_result_BlackbodyRad_p1.idata
idata_BlackbodyRad_p2 = f_bay_result_BlackbodyRad_p2.idata
list_pos_result_BlackbodyRad = [f_bay_result_BlackbodyRad_p1, f_bay_result_BlackbodyRad_p2]
combined_idata_BlackbodyRad, mean_reff_BlackbodyRad = combine_idata(list_pos_result_BlackbodyRad)
combined_idata_BlackbodyRad

Inference data with groups:
	> posterior_predictive
	> log_likelihood
	> observed_data

In [4]:
idata_dict = {
    'Compt': combined_idata_Compt,
    'BlackbodyRad': combined_idata_BlackbodyRad,
}

az.compare(idata_dict, ic='waic', var_name='channels', scale='deviance')

/home/meotor/anaconda3/lib/python3.11/site-packages/arviz/stats/stats.py:1632: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


,rank,elpd_waic,p_waic,elpd_diff,weight,se,dse,warning,scale
BlackbodyRad,0,63.666546,3.128820,0.000000,1.0,9.687126,0.000000,False,deviance
Compt,1,66.005431,4.816576,2.338885,0.0,9.820525,2.595112,True,deviance


In [5]:
loo_Compt = az.loo(combined_idata_Compt, reff=mean_reff_Compt, scale='deviance')
loo_BlackbodyRad = az.loo(combined_idata_BlackbodyRad, reff=mean_reff_BlackbodyRad, scale='deviance')
az.compare({'Compt': loo_Compt,'BlackbodyRad': loo_BlackbodyRad}, ic='loo', scale='deviance')

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
BlackbodyRad,0,63.75059,3.170842,0.00000,1.0,9.700311,0.000000,False,deviance
Compt,1,66.11665,4.872186,2.36606,0.0,9.830203,2.598437,False,deviance
